In [4]:
import os
from zipfile import ZipFile
import pandas as pd
import json
import numpy as np

In [5]:
def extract_json_data(filename = "RAW_META_LUXURY_BEAUTY_20230111.json.zip"):
    separador = os.path.sep
    dir_actual = os.getcwd()
    execution_folder_dir = separador.join(dir_actual.split(separador)[:-1])
    folder_json = os.path.join(execution_folder_dir, "1_Datos", filename)

    path_to_extract = os.path.join(execution_folder_dir, "1_Datos")
    if not os.path.exists(folder_json[:-4]):
        print("hehehe")
        with ZipFile(folder_json, 'r') as f:
            f.extractall(path_to_extract)
    folder_json = folder_json[:-4]
    file_json = os.listdir(folder_json)[0]
    return separador.join([folder_json, file_json])


In [6]:
import pandas as pd
a = pd.DataFrame.from_records([{"a": True, "b": 2},{"a": False, "b": 3}])
print(a.head())

       a  b
0   True  2
1  False  3


In [8]:
def open_json_file(file_json):
    data = []
    with open(file_json, "r") as f:
        for line in f.readlines():
           data.append(json.loads(line))

    df = pd.DataFrame.from_records(data)
    df = df.rename(columns={'asin': 'itemID'})
    return df

def get_data_frame(raw_all_json, raw_meta_json):
    raw_all_df = open_json_file(raw_all_json)
    raw_meta_df = open_json_file(raw_meta_json)

    return raw_all_df, raw_meta_df

In [9]:
raw_all, raw_meta = ["RAW_ALL_BEAUTY_20230111.json.zip", "RAW_META_LUXURY_BEAUTY_20230111.json.zip"]
raw_all_json, raw_meta_json = extract_json_data(raw_all), extract_json_data(raw_meta)
raw_all_df, raw_meta_df = get_data_frame(raw_all_json, raw_meta_json)

In [12]:
print(raw_all_df.columns)
raw_all_df["reviewerID"] = pd.Categorical(raw_all_df["reviewerID"]).codes
raw_all_df["itemID"] = pd.Categorical(raw_all_df["itemID"]).codes
print(raw_meta_df.columns)

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'itemID',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')
Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'details', 'main_cat',
       'similar_item', 'date', 'price', 'itemID', 'imageURL',
       'imageURLHighRes'],
      dtype='object')


In [ ]:
def split_train_test(data: np.ndarray,
                     n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        """
        Line 1: filtramos por cada usuario (u) y nos quedamos con SOLO sus datos.
        Line 2: ordenamos para tener info + reciente al principio.
        """
        user_data = data[data[:, 0] == u]
        sorted_data = user_data[user_data[:, -1].argsort()]

        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:-1])
        else:
            """
            user = u
            Line 1: guardas  (train) user,item de todos menos última ocurrencia.
            Line 2: guardas (test) user,item de la última ocurrencia,
            """
            train_x.append(sorted_data[:-1][:, :-1])
            test_x.append(sorted_data[-1][:-1])
    return np.vstack(train_x), np.stack(test_x)

In [ ]:
data = raw_all_df[['reviewerID', 'itemID', 'unixReviewTime']].astype('int32').to_numpy()

In [ ]:
add_dims=0
for i in range(data.shape[1] - 1):  # do not affect to timestamp
    # MAKE IT START BY 0
    data[:, i] -= np.min(data[:, i])
    # RE-INDEX
    data[:, i] += add_dims
    add_dims = np.max(data[:, i]) + 1
dims = np.max(data, axis=0) + 1 # max de cada columna
#data # item col = items+users

In [ ]:
train_x, test_x = split_train_test(data, dims[0]) # dims[0]: max users

In [3]:
# DELETE FOLDER CREATED
raw_all, raw_meta = ["RAW_ALL_BEAUTY_20230111.json.zip", "RAW_META_LUXURY_BEAUTY_20230111.json.zip"]
import shutil
def delete_folder_raw_data(raw_name):
    path = extract_json_data(raw_name)
    print(path)

    separador = os.path.sep
    folder_to_delete = separador.join(path.split(separador)[:-1])

    print(folder_to_delete)
    shutil.rmtree(folder_to_delete)

delete_folder_raw_data(raw_all)
delete_folder_raw_data(raw_meta)


NameError: name 'extract_json_data' is not defined